In [1]:
#data was cleaned after a first integration run. 
#This is the final integration based on optimized scVI hyperparameters
#CAVE: we used a GPU in Google colab. Be aware that colab environments are difficult to control and dependencies can change anytime. 
#To reproduce this work check the version prints. 

In [2]:
!pip install --quiet scvi-colab
from scvi_colab import install
install()

import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !pip install --quiet scrublet
    import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import anndata
import scvi
import scanpy as sc

sc.set_figure_params(figsize=(4, 4))
scvi.settings.seed = 94705

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

!pip install anndata2ri
import anndata2ri

!pip install session_info

seed=10
sc.logging.print_versions()

     |████████████████████████████████| 237 kB 3.9 MB/s 
     |████████████████████████████████| 51 kB 2.5 MB/s 
INFO     scvi-colab: Installing scvi-tools.                                                                        
INFO     scvi-colab: Install successful. Testing import.                                                           


INFO:pytorch_lightning.utilities.seed:Global seed set to 0
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


     |████████████████████████████████| 647 kB 4.3 MB/s 


INFO:pytorch_lightning.utilities.seed:Global seed set to 94705


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8046 sha256=f419338fda45730b014c05c31300f2422b6da70c258e8c53e40e4de37e0994dd
  Stored in directory: /root/.cache/pip/wheels/5c/1b/4d/111d73980c5c6a8e5e5905a19eccc34296cb359cac54c6c5b9
Successfully built session-info
The `sinfo` package has changed name and is now called `session_info` to become more discoverable and self-explanatory. The `sinfo` PyPI package will be kept around to avoid breaking old installs and you can downgrade to 0.3.2 if you want to use it without seeing this message. For the latest features and bug fixes, please install `session_info` instead. The usage and defaults also changed slightly, so please review the latest README at https://gitlab.com/joelostblom/session_info.
-----
anndata    

In [3]:
path = '.../Atlas6.1_3000HVG.h5ad'

In [5]:
adata = scvi.data.read_h5ad(path)

In [ ]:
adata

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata, 
    batch_key="species",
    layer="counts", categorical_covariate_keys=["orig_ident"], 
    continuous_covariate_keys=["percent_mt"])
model = scvi.model.SCVI(adata)
model
vae = scvi.model.SCVI(adata, n_layers=3, n_latent=30, gene_likelihood="nb", dropout_rate=0.1)
vae.train(max_epochs = 600, plan_kwargs={"lr":0.001}, early_stopping = True, early_stopping_patience = 15)
model = vae

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 378/600:  63%|██████▎   | 378/600 [12:32:35<7:21:59, 119.46s/it, loss=801, v_num=1]
Monitored metric elbo_validation did not improve in the last 15 records. Best score: 798.321. Signaling Trainer to stop.


In [ ]:
model.save(".../Model_Atlas6.1_V2")
